In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [77]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(768,)))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(
    loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(), metrics=['accuracy']
)

In [78]:
MAX = 1000

def prepare_output(array):
    result = []
    for element in array:
        element = min(MAX, max(-MAX, element))
        element = 0.5 + 0.5*element/MAX
        result.append([element, 1 - element])
    return np.array(result)


In [79]:
df = pd.read_csv('data1.csv')
data, evaluations = df.iloc[:,:-1], df.iloc[:,-1:]
# print(data.shape)
# print(data.to_numpy())

# x_train = x_train.reshape(60000, 784).astype("float32") / 255
# x_test = x_test.reshape(10000, 784).astype("float32") / 255

# y_train = y_train.astype("float32")
# y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = data[:int(len(data)*0.1)].to_numpy()
y_val = prepare_output(evaluations[:int(len(data)*0.1)].to_numpy().flatten())
x_train = data[int(len(data)*0.1):int(len(data)*0.9)].to_numpy()
y_train = prepare_output(evaluations[int(len(data)*0.1):int(len(data)*0.9)].to_numpy().flatten())
x_test = data[int(len(data)*0.9):].to_numpy()
y_test = prepare_output(evaluations[int(len(data)*0.9):].to_numpy().flatten())
print(x_val.shape)
print(y_val.shape)
# print(x_train.shape)
# print(x_test.shape)


(249, 768)
(249, 2)


In [80]:
history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=30,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Epoch 1/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6921 - accuracy: 0.7617 - val_loss: 0.6908 - val_accuracy: 0.9357
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6832 - accuracy: 0.8871 - val_loss: 0.6908 - val_accuracy: 0.9357
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6812 - accuracy: 0.9137 - val_loss: 0.6910 - val_accuracy: 0.9357
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6800 - accuracy: 0.9187 - val_loss: 0.6914 - val_accuracy: 0.9357
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6795 - accuracy: 0.9127 - val_loss: 0.6906 - val_accuracy: 0.9237
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6787 - accuracy: 0.9172 - val_loss: 0.6907 - val_accuracy: 0.9357
Epoch 7/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6780 - accuracy: 0.9288 - val_loss: 0.6911 - val_accuracy: 0.9357
Epoch 8/30
16/16 [=

In [81]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
2/2 [==============================] - 0s 4ms/step - loss: 0.6856 - accuracy: 0.9520
test loss, test acc: [0.6856420636177063, 0.9520000219345093]
Generate predictions for 3 samples
1/1 [==============================] - 0s 79ms/step
predictions shape: (3, 2)
